In [13]:
import os
import pandas as pd
import torch.nn as nn
import torch

# Data Pre-Processing

### Transfroming the data into more readable input to the model

In [32]:
def encode_song(song):
    result = []
    prev = {'note0': -1, 'note1': -1, 'note2': -1, 'note3': -1} # COME BACKKKK
    for index, row in song.iterrows():
        frame = []
        for voice in ['note0', 'note1', 'note2', 'note3']:
            pitch = row[voice]
            previous_pitch = prev[voice]
            
            tied = 1 if pitch == previous_pitch else 0
            frame.append((int(pitch), tied))
            prev[voice] = pitch
        result.append(frame)
        result.append("|||")
        print(result)
    tensor = torch.tensor(result, dtype=torch.int64)
    return result

In [33]:
folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                song = encode_song(df)
                if dirname == 'test':
                    test.append(song)
                if dirname == 'train':
                    train.append(song)
                if dirname == 'valid':
                    validation.append(song)

[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||']
[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||']
[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||']
[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||']
[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(69, 0), (65, 0), (60, 0), (53, 0)], '|||']
[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(65, 1), (60, 1), (57, 1), (41, 1)], '|||', [(69, 0), (65, 0), (60, 0), (53, 0)], '|||', [(69, 1), (65, 1), (60, 1), (53, 1)], '|||']
[[(65, 0), (60, 0), (57, 0), (41, 0)], '|||', [(6

ValueError: expected sequence of length 4 at dim 1 (got 3)